# Lab 09 - High Performance Computing
#### Module Imports

In [6]:
import multiprocessing as mproc, numpy, time

## Exercise 01 - Number of Processors

In [2]:
print("Number of processors:", mproc.cpu_count())

Number of processors: 8


## Exercise 02 - Array Normalization

In [3]:
def create_list(rows, cols, min, max):
    return numpy.random.randint(min, max, size=[rows, cols])

In [12]:
def normalize_by_row(lst):
    ROWS, COLS = lst.shape
    norm_list = numpy.zeros((ROWS, COLS))

    for row in range(ROWS):
        row_slice = lst[row][:]
        ROW_MIN = min(row_slice)
        ROW_MAX = max(row_slice)

        for col in range(COLS):
            norm_list[row][col] = (row_slice[col] - ROW_MIN) / (ROW_MAX - ROW_MIN)

    return norm_list

In [13]:
MIN = 0
MAX = 10
SIZES = [ 100, 500, 1000, 2000, 3000 ]

for size in SIZES:
    START = time.time()
    lst = create_list(size, size, MIN, MAX)
    norm_list = normalize_by_row(lst)
    print("Sequential normalization by row for size", size, "took", time.time() - START, "seconds")

Sequential normalization by row for size 100 took 0.009999275207519531 seconds
Sequential normalization by row for size 500 took 0.23001551628112793 seconds
Sequential normalization by row for size 1000 took 0.8617680072784424 seconds
Sequential normalization by row for size 2000 took 3.4399664402008057 seconds
Sequential normalization by row for size 3000 took 7.78710675239563 seconds
